<a href="https://colab.research.google.com/github/Thiago-TP/Trabalhos_de_Software_Basico/blob/main/TrabalhoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conversão de Label das imagens

import os
import shutil
import random

# Diretórios base de imagens, rótulos e destino final do dataset
img_dir = '/content/all_images'        # Diretório com imagens originais
label_dir = '/content/all_labels'      # Diretório com todos os rótulos
dataset_dir = '/content/dataset'       # Diretório da nova estrutura do dataset

# Tamanho padrão das imagens
img_size = 512

# Quantidade de imagens em cada subdivisão do dataset
train_count = 132  # Quantidade para treino
val_count = 44     # Quantidade para validação
test_count = 44    # Quantidade para teste

# Criar as pastas para armazenar as imagens e os rótulos divididos
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(dataset_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels', split), exist_ok=True)

# Listar todas as imagens disponíveis no diretório
all_images = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
all_images.sort()  # Ordena para garantir consistência na seleção
random.seed(42)    # Define uma semente para reprodução dos resultados
random.shuffle(all_images)  # Embaralha as imagens aleatoriamente

# Divide a lista de imagens conforme os contadores definidos
train_imgs = all_images[:train_count]
val_imgs = all_images[train_count:train_count + val_count]
test_imgs = all_images[train_count + val_count:]  # O restante vai para teste

# Dicionário com a separação por tipo de uso
splits = {
    'train': train_imgs,
    'val': val_imgs,
    'test': test_imgs
}

# Converte rótulos no formato (xmin, ymin, xmax, ymax) para o formato YOLO
def convert_label(line):
    parts = list(map(int, line.strip().split()))  # Lê os números da linha e converte para inteiros

    # Caso o rótulo não tenha classe, assume classe 0
    if len(parts) == 4:
        xmin, ymin, xmax, ymax = parts
        class_id = 0
    elif len(parts) == 5:
        class_id, xmin, ymin, xmax, ymax = parts
    else:
        return None  # Ignora linhas que não seguem os formatos esperados

    # Conversão para o formato YOLO: centro X, centro Y, largura e altura (normalizados)
    x_center = (xmin + xmax) / 2 / img_size
    y_center = (ymin + ymax) / 2 / img_size
    width = (xmax - xmin) / img_size
    height = (ymax - ymin) / img_size

    # Retorna a linha no formato YOLO (classe x y w h)
    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# Percorre todas as divisões (treino, validação, teste)
for split, image_list in splits.items():
    for img_name in image_list:
        base = os.path.splitext(img_name)[0]  # Nome do arquivo sem extensão

        # Caminhos de origem da imagem e do rótulo
        img_src = os.path.join(img_dir, img_name)
        lbl_src = os.path.join(label_dir, base + '.txt')

        # Caminhos de destino da imagem e do rótulo convertidos
        img_dst = os.path.join(dataset_dir, 'images', split, img_name)
        lbl_dst = os.path.join(dataset_dir, 'labels', split, base + '.txt')

        # Copia a imagem para o novo diretório
        shutil.copy(img_src, img_dst)

        # Abre o arquivo de rótulo original e escreve o novo rótulo no formato YOLO
        with open(lbl_src, 'r') as infile, open(lbl_dst, 'w') as outfile:
            for line in infile:
                yolo_line = convert_label(line)
                if yolo_line:
                    outfile.write(yolo_line + '\n')  # Escreve a linha convertida


In [ ]:
# Instala o pacote 'ultralytics', que contém implementações modernas do modelo YOLO,
!pip install ultralytics

# Verifica se uma GPU com suporte CUDA está disponível no sistema.
import torch
print("CUDA disponível:", torch.cuda.is_available())


In [ ]:
# Importa a classe principal da biblioteca Ultralytics, que oferece suporte aos modelos YOLO
from ultralytics import YOLO

# Carrega o modelo YOLOv8 na versão 'n' (nano), versão leve e rápida
model = YOLO('yolov8n.pt')

# Inicia o treinamento do modelo com os seguintes parâmetros:
model.train(
    data='/content/dataset/data.yaml',  # Caminho para o arquivo de configuração do dataset
    epochs=2000,                        # Número de épocas de treinamento
    imgsz=512                           # Tamanho das imagens
)

In [ ]:
# Avalia o desempenho do modelo treinado usando o conjunto de teste definido no arquivo data.yaml
metrics = model.val(split='test')

# Imprime a métrica AP50
print("AP50:", metrics.box.map50)


In [ ]:
# Importa a classe YOLO da biblioteca Ultralytics
from ultralytics import YOLO

# Importa a biblioteca os para manipulação de diretórios
import os

# Carrega o modelo treinado anteriormente, usando o melhor checkpoint salvo durante o treinamento
model = YOLO('runs/detect/train4/weights/best.pt')

# Define o diretório onde estão as imagens que serão usadas para teste/predição
test_images_dir = '/content/dataset/images/test'

# Realiza predições sobre as imagens da pasta de teste
results = model.predict(
    source=test_images_dir,  # Caminho para as imagens de entrada
    imgsz=512,               # Redimensiona as imagens para 512x512 durante a inferência
    save=True,               # Salva as imagens com as caixas de detecção desenhadas
    save_txt=True,           # Salva os rótulos das predições em arquivos .txt (formato YOLO)
    conf=0.25                # Confiança mínima para mostrar uma predição (ex: se for menor que 25%, é ignorada)
)


In [ ]:
import shutil

# Define o caminho da pasta que contém os resultados das predições (imagens e .txt)
pasta = '/content/runs/detect/predict'  # Pasta gerada após model.predict()

# Cria um arquivo ZIP compactando toda a pasta de predições
shutil.make_archive('/content/predicoes', 'zip', pasta)

from google.colab import files

# Faz o download do arquivo ZIP para o computador do usuário
files.download('/content/predicoes.zip')